In [ ]:
import math
import numpy as np
import pandas as pd
from datetime import datetime
import time
from pyspark.sql.types import *
import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from pyspark.sql.types import *
from pyspark.sql import functions 
from tensorflow.keras import layers
import tensorflow as tf

In [ ]:
customSchema = StructType([StructField("age", FloatType(), True), 
                          StructField("job", FloatType(), True), 
                          StructField("marital", FloatType(), True), 
                          StructField("education", FloatType(), True),
                          StructField("default", FloatType(), True),
                          StructField("balance", FloatType(), True),
                          StructField("housing", FloatType(), True),
                          StructField("loan", FloatType(), True),
                          StructField("contact", FloatType(), True),
                          StructField("day_of_week", FloatType(), True),
                          StructField("month", FloatType(), True),
                          StructField("duration", FloatType(), True),
                          StructField("campaign", FloatType(), True),
                          StructField("pdays", FloatType(), True),
                          StructField("previous", FloatType(), True),
                          StructField("poutcome", FloatType(), True),
                          StructField("y", FloatType(), True)])

df = spark.read.load('hdfs://localhost:9000/user1/Bank-Pyspark1.csv', format="csv", header="true", sep=',', schema=customSchema)

In [ ]:
type(df)

In [ ]:
df.printSchema()

In [ ]:
df.show(20)

In [ ]:
# create model
model = Sequential()

model.add(Dense(500, activation='sigmoid')) # 500 neurons
model.add(Dense(1, activation='sigmoid')) # 1 output neuron

In [ ]:
# Compile model
X_columns = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day_of_week', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']
Y_column = 'y'  # Replace with the actual label column name

# Extract X and Y
X_spark = df.select(X_columns)
Y_spark = df.select(Y_column)

# # Convert to NumPy arrays
X = np.array(X_spark.collect())  # Collecting to local NumPy array
Y = np.array(Y_spark.collect()).flatten()  # Flatten Y since it's a single column
X = np.array(X, dtype=np.float32)
Y = np.array(Y, dtype=np.float32) 


optimizer = Adam(clipvalue=0.5, learning_rate=1e-5)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# # Fit the model
model.fit(X, Y, epochs=10, batch_size=50)# 150 epoch, 10 batch size, verbose = 2

In [ ]:
# calculate predictions
predictions = model.predict(X)    # predicting Y only using X
# Round predictions
#rounded = [int(numpy.round(x, 0)) for x in predictions]
rounded = [round(x[0]) for x in predictions]
scores = model.evaluate(X, Y)

actual = Y
predicted = rounded

confusion_matrix = metrics.confusion_matrix(actual, predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show() 